In [8]:
import os
import json
import pandas as pd
import langchain
import langchain_groq
import traceback

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [74]:
from operator import itemgetter

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser




In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(
    temperature=0,
    model="mixtral-8x7b-32768",
     api_key=os.getenv("GROQ_API_KEY") # Optional if not set as an environment variable
)

system = "You are a witty assistant that likes giving relevant analogies to explain queries."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Explain the importance of low latency for LLMs."})

AIMessage(content="Sure, I'd be happy to explain!\n\nThink of low latency in large language models (LLMs) as the difference between a quick-witted conversation partner and a slow, plodding one. In a fast-paced conversation, you want your partner to keep up with the flow of ideas and respond promptly. If they take too long to think and respond, the conversation can feel stilted and unnatural.\n\nSimilarly, in the world of LLMs, low latency is crucial for maintaining the flow of interaction. When a model takes too long to generate a response, it can disrupt the user's train of thought and make the interaction feel less fluid and natural.\n\nMoreover, in many applications, low latency is not just a matter of convenience, but a necessity. For example, in real-time chatbots or voice assistants, users expect quick and snappy responses. If the model takes too long to generate a response, it can lead to a poor user experience and even frustration.\n\nTherefore, just as in a lively conversation

In [14]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [25]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [18]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [77]:
quiz_chain= quiz_generation_prompt | chat | StrOutputParser()

quiz_chain.invoke({})

In [20]:
in_text= """Advanced topics in Newtonian physics at the undergraduate level typically include:
Lagrangian and Hamiltonian Formulations: These mathematical frameworks provide alternative ways to describe classical mechanics, emphasizing energy and momentum conservation. They are essential for understanding more complex systems and are often used in advanced courses.
Deterministic Chaos Theory: This topic explores the intricate behavior of complex systems, such as the Poincaré surface-of-section, which helps students visualize and analyze the dynamics of chaotic systems.
Euler's Rigid Body Equations of Motion: These equations describe the motion of rigid bodies, which is crucial for understanding many real-world phenomena, such as the motion of spacecraft and the behavior of molecules.
Canonical Perturbation Theory: This method is used to study the long-term behavior of complex systems by analyzing small perturbations to their motion. It is essential for understanding many phenomena in astrophysics and cosmology.
Central Force Motion and Scattering: These topics involve the study of the motion of particles under the influence of central forces, such as gravitational or Coulomb forces. They are fundamental to understanding many astrophysical and cosmological phenomena.
These advanced topics in Newtonian physics provide a deeper understanding of the underlying principles and help students develop a more nuanced appreciation for the complexities of classical mechanics."""

In [29]:
dict_passed_in={"text":in_text, "number":5 , "subject":"physics undergrads", "tone":"moderate", "response_json":RESPONSE_JSON}

In [81]:
str_json_output =(quiz_chain.invoke(dict_passed_in)[14:])

In [82]:
from datetime import datetime

datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'06_19_2024_12_43_29'

In [48]:
quiz_chain_w_output_key= RunnablePassthrough.assign(quiz= quiz_chain)

In [61]:
response=quiz_chain_w_output_key.invoke(dict_passed_in)

In [67]:
str_parser = StrOutputParser()
parsed_output = str_parser.parse(response.get("quiz"))



content="RESPONSE\\_JSON\n{'1': {'mcq': 'Which of the following mathematical frameworks provides an alternative way to describe classical mechanics, emphasizing energy and momentum conservation?', 'options': {'a': 'Lagrangian Formulation', 'b': 'Hamiltonian Formulation', 'c': 'Newtonian Formulation', 'd': 'Eulerian Formulation'}, 'correct': 'b'}, '2': {'mcq': 'Which of the following topics studies the intricate behavior of complex systems, such as the Poincaré surface-of-section?', 'options': {'a': 'Lagrangian and Hamiltonian Formulations', 'b': 'Deterministic Chaos Theory', 'c': 'Euler\\'s Rigid Body Equations of Motion', 'd': 'Canonical Perturbation Theory'}, 'correct': 'b'}, '3': {'mcq': 'Which of the following equations describe the motion of rigid bodies?', 'options': {'a': 'Lagrangian Equations', 'b': 'Hamiltonian Equations', 'c': 'Euler\\'s Rigid Body Equations of Motion', 'd': 'Canonical Transformation Equations'}, 'correct': 'c'}, '4': {'mcq': 'Which of the following methods i

<class 'langchain_core.messages.ai.AIMessage'>


In [66]:
print(type(final_quiz))

<class 'langchain_core.messages.ai.AIMessage'>


In [35]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [49]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [60]:
full_prompt_chain= (
    quiz_chain_w_output_key
    |{
        "quiz":itemgetter("quiz"),
        "subject":itemgetter("subject")
    }
    | quiz_evaluation_prompt
    |chat
    |RunnablePassthrough.assign
)

NameError: name 'Runna' is not defined

In [59]:
full_prompt_chain.invoke(dict_passed_in)

AIMessage(content="The complexity of the quiz is moderate to high, suitable for advanced undergraduates. However, the tone can be updated to be more student-friendly. Here are the revised questions:\n\n1. Which of these math tools offers a unique take on classical mechanics, focusing on energy and momentum conservation?\n   a) Lagrangian Formulation\n   b) Hamiltonian Formulation\n   c) Newtonian Formulation\n   d) Eulerian Formulation\n2. Which area of study dives into the intricate patterns of complex systems, like the Poincaré surface-of-section?\n   a) Lagrangian and Hamiltonian Formulations\n   b) Deterministic Chaos Theory\n   c) Euler's Rigid Body Equations of Motion\n   d) Canonical Perturbation Theory\n3. Which set of equations guides the motion of rigid bodies?\n   a) Lagrangian Equations\n   b) Hamiltonian Equations\n   c) Euler's Rigid Body Equations of Motion\n   d) Canonical Transformation Equations\n4. Which method helps analyze the long-term behavior of complex systems 